### **Schritt 1: Importieren der Abhängigkeiten**

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### **Schritt 2: Laden der Daten**

In [2]:
# die Pfade zu den Dateien zu laden
movies_path = "https://raw.githubusercontent.com/andresmorenoviteri/workshops/refs/heads/main/movies.csv"
ratings_path = "https://raw.githubusercontent.com/andresmorenoviteri/workshops/refs/heads/main/ratings.csv"

In [3]:
movies_df = pd.read_csv(movies_path)
ratings_df = pd.read_csv(ratings_path)

### **Schritt 3: Datenexploration** 

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df.head()

In [ ]:
n_ratings = len(ratings_df)
n_movies = ratings_df["movieId"].nunique()
n_users = ratings_df['userId'].nunique()

print(f"number of ratings: {n_ratings}")
print(f"movies rated: {n_movies}")
print(f"users: {n_users}")
print(f"average number of ratings per movie: {round(n_ratings/n_movies, 2)}")
print(f"average number of ratings per user: {round(n_ratings/n_users, 2)}") # very high number, not realistic, usually only a couple of ratings

### Verteilung der Filmbewertungen

In [ ]:
# Sind die Bewertungen gleichmäßig verteilt oder in irgendeiner Weise verzerrt?
sns.countplot(x='rating', hue='rating', data=ratings_df, palette='viridis')

In [ ]:
print(f"mean global rating: {round(ratings_df['rating'].mean(), 2)}")

mean_rating = ratings_df.groupby('userId')['rating'].mean()
print(f"mean rating per user: {round(mean_rating.mean(), 2)}")

In [ ]:
ratings_df.describe()

### Welche Filme werden am häufigsten bewertet?

In [ ]:
ratings_df['movieId'].value_counts()

In [ ]:
movie_ratings_df = ratings_df.merge(movies_df, how='inner', on='movieId')
movie_ratings_df['title'].value_counts()

In [ ]:
# prüfen, ob in den Datensätzen Leerstellen vorhanden sind
movie_ratings_df.info()

Forrest Gump, Shawshank redemption und Pulp Fiction sind die meistbewerteten Filme

### Welches sind die am schlechtesten und am besten bewerteten Filme?

In [ ]:
movie_ratings_df.head()

In [ ]:
movie_stats = movie_ratings_df.groupby('title')['rating'].agg(['count', 'mean']).reset_index()
movie_stats.head()

In [ ]:
lowest_rated = movie_stats.loc[movie_stats['mean'].idxmin()]['title']
print(f'lowest rated movie: {lowest_rated}')

In [ ]:
highest_rated = movie_stats.loc[movie_stats['mean'].idxmax()]['title']
print(f'highest rated movie: {highest_rated}')

In [ ]:
# Anzahl der Bewertungen für Salem's Lot überprüfen 
len(movie_stats[movie_stats['title'] == "'Salem's Lot (2004)"])

eine einzige Bewertung reicht nicht aus, um einen hoch bewerteten Film zu erhalten, verwenden Sie den Bayes'schen Durchschnitt, um eine bessere Vorhersage zu erhalten

**Bayesianischer Durchschnitt**

Der Bayesianische Durchschnitt ist eine Methode zur Schätzung des Mittelwerts einer Grundgesamtheit unter Verwendung externer Informationen, insbesondere einer bereits bestehenden Überzeugung, die in die Berechnung einfließt.

Der Bayesianische Durchschnitt ist definiert als:

$x_{i} =  \frac{C \times m + \Sigma{\text{ratings}}}{C+N}$
 
- $C$ wird auf der Grundlage der Größe des Datensatzes gewählt, in unserem Fall ist es die durchschnittliche Anzahl der Bewertungen für jeden Film.
- $m$ ist der vorherige Mittelwert, in unserem Fall also die durchschnittliche Bewertung eines Films.
- $N$ ist die Gesamtzahl der Bewertungen für den Film $i$.

In [ ]:
c = movie_stats['count'].mean()
m = movie_stats['mean'].mean()

print(f'average number of ratings for a given movie: {round(c, 2)}')
print(f'average rating for a given movie: {round(m, 2)}')

# create new column with Bayesian averages
bayesian_average_list = []
for idx, row in movie_stats.iterrows():
    x = ((c*m)+(row['count']*row['mean']))/((c)+row['count'])
    bayesian_average_list.append(x)

In [ ]:
movie_stats['bayesian_average'] = bayesian_average_list
movie_stats.head()

In [ ]:
movie_stats[movie_stats['title'].str.contains("Salem's Lot")]

sehen wir, dass „Salem's Lot“ nicht mehr der am besten bewertete Film ist

In [ ]:
bayesian_min = movie_stats.loc[movie_stats['bayesian_average'].idxmin()]['title']
bayesian_min

In [ ]:
bayesian_max = movie_stats.loc[movie_stats['bayesian_average'].idxmax()]['title']
bayesian_max

In [ ]:
movie_stats.sort_values(by='bayesian_average', ascending=False)

Die bestbewerteten Filme sind Shawshank Redemption, Godfather und Fight Club

Die am schlechtesten bewerteten Filme sind Speed 2, Battlefield Earth und Godzilla

### Ein Blick auf Filmgenres

In [ ]:
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : x.split('|'))
movies_df.head()

**Filmgenres zählen**

In [ ]:
from collections import Counter

genres_list = movies_df['genres'].explode()
genre_counts = Counter(genres_list)
genre_counts

In [ ]:
print(f"the 5 most common genres are: {genre_counts.most_common(5)}")

In [ ]:
genre_counts_df = pd.DataFrame([genre_counts]).T.reset_index()
genre_counts_df.columns = ['genre', 'count']

sns.barplot(x='genre', y='count', hue='genre', data=genre_counts_df.sort_values(by='count', ascending=False), legend=False, palette='viridis')
plt.xticks(rotation=90)
plt.show()

### **Schritt 4: Vorverarbeitung der Daten**

Wir werden die Technik der kollaborativen Filterung anwenden, um Empfehlungen für die Benutzer zu erstellen.
Die Grundlage für diese Technik ist, dass ähnliche Nutzer einen ähnlichen Filmgeschmack haben.

Wir beginnen damit, unsere Daten in eine „Nutzwertmatrix“ umzuwandeln, d. h. eine Matrix aus Nutzern und Objekten. Die Zeilen werden repräsentieren die Nutzer, während die Spalten die Filme darstellen.

In [ ]:
movie_ratings_df.head()

In [ ]:
movie_stats['count'][movie_stats['title'] == 'Toy Story (1995)']

In [32]:
from scipy.sparse import csr_matrix

def create_matrix(df):

    rows = df['userId'].nunique()
    columns = df['movieId'].nunique()

    userMapper = dict(zip(np.unique(df['userId']), list(range(rows))))
    movieMapper = dict(zip(np.unique(df['movieId']), list(range(columns))))

    invUserMapper = dict(zip(list(range(rows)), np.unique(df['userId'])))
    invMovieMapper = dict(zip(list(range(columns)), np.unique(df['movieId'])))

    # create the user-item interaction matrix
    userMovieMatrix = pd.pivot_table(df, values='rating', index='userId', columns='movieId').fillna(0)
    sparseMatrix= csr_matrix(userMovieMatrix)

    # convert matrx to sparse matrix
    return sparseMatrix, userMapper, movieMapper, invUserMapper, invMovieMapper 

sparse_matrix, user_mapper, movie_mapper, inv_user_mapper, inv_movie_mapper = create_matrix(movie_ratings_df)

In [ ]:
sparse_matrix.nnz

### Maß für die Dünnbesetztheit
Die Dünnbesetzt wird berechnet, indem die Anzahl der gespeicherten (nicht leeren) Felder durch die Gesamtzahl der Felder geteilt wird. Die Anzahl der gespeicherten Werte in unserer Matrix entspricht der Anzahl der Bewertungen in unserem Datensatz.

In [ ]:
n_total = sparse_matrix.shape[0] * sparse_matrix.shape[1]
num_ratings = sparse_matrix.nnz #csr_matrix.nnz counts the stored values in the sparse matrix. The rest of the cells are empty
sparsity = num_ratings / n_total
print(f"Matrix sparsity: {round(sparsity*100, 2)}%")

Das **Kaltstartproblem** tritt auf, wenn es neue Nutzer und Filme gibt, die noch keine Bewertungen haben.
 
In unserem Movielens-Datensatz haben alle Nutzer und Filme mindestens eine Bewertung, aber im Allgemeinen ist es wichtig, zu prüfen, ob Nutzer und Filme wenige oder keine Interventionen haben


hier wird überprüft, ob alle Zellen im Datensatz nicht leer sind

In [ ]:
n_ratings_per_user = sparse_matrix.getnnz(axis=1)
len(n_ratings_per_user)

In [ ]:
print(f"The most active user rated {n_ratings_per_user.max()} movies")
print(f"The least active user rated {n_ratings_per_user.min()} movies")

In [ ]:
n_ratings_per_movie = sparse_matrix.getnnz(axis=0)
len(n_ratings_per_movie)

In [ ]:
print(f"The most rated movie has {n_ratings_per_movie.max()} ratings")
print(f"The least rated movie has {n_ratings_per_movie.min()} ratings")

In [ ]:
plt.figure(figsize=(16, 4))
plt.subplot(1, 2, 1)
sns.kdeplot(n_ratings_per_user, fill=True)
plt.xlim(0)
plt.title("Number of ratings per user", fontsize=14)
plt.xlabel("number of ratings per user")
plt.ylabel("density")
plt.subplot(1, 2, 2)
sns.kdeplot(n_ratings_per_movie, fill=True)
plt.xlim(0)
plt.title("Number of ratings per movie", fontsize=14)
plt.xlabel("number of ratings per movie")
plt.ylabel("density")
plt.show()

### **Schritt 5: Item-Empfehlungen mit *K-Nearest-Neighbor***

Wir werden die k Filme finden, die die ähnlichsten Vektoren für das Nutzerengagement für einen Film i haben.


In [59]:
from sklearn.neighbors import NearestNeighbors

def find_recommended_movies(movieId, sparseMatrix, movieMapper, invMovieMapper, k, metric='cosine'):
    # transpose matrix to get movie recommendations from matrix
    sparseMatrix = sparseMatrix.T
    recommendationIds = []
    movieIdx = movieMapper[movieId]
    movieVec = sparseMatrix[movieIdx]
    if isinstance(movieVec, (np.ndarray)):
        movieVec = movieVec.reshape(1, -1)
    # use k+1 since knn outputs include the movieId of interest
    knn = NearestNeighbors(n_neighbors=k+1, metric=metric, algorithm='brute')
    knn.fit(sparseMatrix)
    recommendations = knn.kneighbors(movieVec, return_distance=False)
    for i in range(0, k):
        n = recommendations.item(i)
        recommendationIds.append(invMovieMapper[n])
    recommendationIds.pop(0)
    return recommendationIds

In [ ]:
movie_mapper[33794]

### Interaktive Grafik

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

def interactive_visualization(sparseMatrix, movieMapper, invMovieMapper, k, metric='cosine'):
    # Transpose sparse matrix
    sparseMatrix = sparseMatrix.T

    # Normalize before dimensionality reduction
    scaler = StandardScaler()
    scaledSparseMatrix = scaler.fit_transform(sparseMatrix.toarray())  

    # Apply t-SNE for better visualization
    tsne = TSNE(n_components=2, perplexity=50, random_state=42)
    reducedMatrix = tsne.fit_transform(scaledSparseMatrix)

    # Create a DataFrame for visualization
    df = pd.DataFrame(reducedMatrix, columns=['PCA1', 'PCA2'])
    df['movieId'] = [invMovieMapper[i] for i in range(sparseMatrix.shape[0])]

    # Map movieId to titles efficiently
    movie_id_to_title = movie_ratings_df.set_index('movieId')['title'].to_dict()
    df['title'] = df['movieId'].map(movie_id_to_title)

    # Fit KNN for recommendations
    knn = NearestNeighbors(n_neighbors=k+1, metric=metric, algorithm='brute')
    knn.fit(sparseMatrix)

    def get_recommendations(movieId):
        movieIdx = movieMapper[movieId]
        movieVec = sparseMatrix[movieIdx]
        movieVec = movieVec.reshape(1, -1) if isinstance(movieVec, np.ndarray) else movieVec
        recommendations = knn.kneighbors(movieVec, return_distance=False)[0][1:]  # Exclude the movie itself
        return [invMovieMapper[n] for n in recommendations]

    # Generate recommendations
    df['recommendations'] = df['movieId'].apply(lambda x: [movie_id_to_title.get(int(i), "Unknown") for i in get_recommendations(x)])

    # Assign unique color values
    df['color'] = range(len(df))

    # Create an interactive scatter plot
    fig = px.scatter(
        df,
        x='PCA1',
        y='PCA2',
        hover_data=['title', 'recommendations'],
        title="Movie Clusters with Recommendations",
        template='plotly_dark',
        color='color',
        opacity=0.6
    )

    # Improve marker visibility
    fig.update_traces(marker=dict(size=4, opacity=0.7))
    fig.update_layout(clickmode='event+select')

    # Show the plot
    fig.show()

# Call the function with appropriate parameters
interactive_visualization(sparse_matrix, movie_mapper, inv_movie_mapper, 5)

**find_recommended_movies()** liefert eine Liste von **movieIds**, die dem gewünschten Film am ähnlichsten sind.

Wir erstellen ein Wörterbuch, das die **movieId** dem **title** des Films zuordnet.

In [ ]:
movie_titles = find_recommended_movies(33794, sparse_matrix, movie_mapper, inv_movie_mapper, 10) 
movie_titles

In [ ]:
# aus den gefundenen Indizes, müssen wir diese in die dazu passende movieId umwandeln. 
print(f"for the movie {movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == 33794].unique().item()}, the recommendations are:")
for id in movie_titles:
    print(movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == id].unique().item())

für den Film Toy Story scheinen die Empfehlungen alle Filme aus den 90er Jahren zu sein, was logisch erscheint

wir könnten auch andere Abstandsmetriken neben *Kosinus* ausprobieren, wie *Manhattan* oder *Euklidisch*.

*Kosinus* ist eine gute Metrik, wenn wir nicht die Größe der Vektoren für den Abstand in Betracht ziehen wollen, sondern eher die Richtung des Vektors

Bei der Verwendung von *Manhattan*- oder *Euklidischen*-Distanzen ist es wichtig, die Daten vor ihrer Anwendung zu normalisieren, wenn die verschiedenen Parameter unserer Daten unterschiedliche Bereiche aufweisen.

die beste Praxis in diesem Schritt zur Optimierung der Hyperparameter ist die Durchführung von Gittersuchen, insbesondere die zufällige Gittersuche

In [ ]:
movie_titles = find_recommended_movies(1, sparse_matrix, movie_mapper, inv_movie_mapper, 10, metric='euclidean') 
movie_titles

In [ ]:
# aus den gefundenen Indizes müssen wir diese in die dazu passende movieId umwandeln 
print(f"for the movie {movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == 1].unique().item()}, the recommendations are:")
for id in movie_titles:
    print(movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == id].unique().item())

Bei Verwendung des euklidischen Abstands ergeben sich einige verschiedene Filme

### **Schritt 6: Behandlung des Kaltstartproblems**

Die kollaborative Filterung stützt sich ausschließlich auf die Interaktionen zwischen Nutzern und Objekten innerhalb der Nutzwertmatrix. Das Problem bei diesem Ansatz besteht darin, dass ein neuer Benutzer oder ein neues Element aus der Nutzenmatrix ausgeschlossen wird, sobald es einen solchen gibt. Dies ist als **Kaltstartproblem** bekannt. Die inhaltsbasierte Filterung ist eine Möglichkeit, dieses Problem zu umgehen, da sie sich auf die Erstellung von Empfehlungen auf der Grundlage von Benutzer- und Artikelmerkmalen konzentriert.

Wir beginnen mit der Umwandlung einer *Genre*-Spalte in binäre Merkmale. Jedes Genre hat seine eigene Spalte und wird mit 1en und 0en gefüllt.

In [ ]:
movies_df = pd.read_csv(movies_path)
movies_df.head()

In [ ]:
movie_genres = movies_df['genres'].str.get_dummies(sep='|')
movie_genres.head()

In [ ]:
movie_genres.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(movie_genres)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

Die *cosine_similarity* ermittelt einen Wert zwischen 0 und 1, der angibt, wie ähnlich jede Zeile der einen Matrix den anderen Zeilen der zweiten Matrix ist.

1 bedeutet identisch und 0 ist völlig unterschiedlich. 

Da wir ursprünglich 9742 Filme haben, ist es richtig, dass die Form dieser Matrix wie folgt aussehen würde $(n_{\text{movies}}, n_{\text{movies}})$

#### Erstellen einer Filmfindungsfunktion

Wenn wir Filmempfehlungen ähnlich wie Jumanji erhalten möchten, müssen wir den genauen Titel kennen, wie er in unserem Datensatz erscheint. Jumanji ist zum Beispiel als „Jumanji (1995)“ aufgeführt. Wenn wir ihn falsch schreiben oder das Erscheinungsjahr vergessen, erkennt der Empfehlungsgeber den Film nicht.

Um es den Nutzern leichter zu machen, können wir das Python-Paket fuzzywuzzy verwenden. Es hilft dabei, den Titel zu finden, der am ehesten mit einer vom Benutzer eingegebenen Zeichenkette übereinstimmt. Wir werden eine Funktion, movie_finder(), erstellen, die fuzzywuzzy's String-Matching verwendet, um die beste Übereinstimmung für den Eingabetitel vorzuschlagen.

In [ ]:
from fuzzywuzzy import process

def movie_finder(title):
    all_movies = movies_df['title'].tolist()
    closest_match = process.extractOne(title, all_movies)
    return closest_match[0]

test mit jumanji

In [ ]:
title = movie_finder('jumaidi')
title

um eine Empfehlung zu erhalten, erstellen wir einen Film-Mapper für den Index in der Matrix

In [ ]:
movie_idx = dict(zip(movies_df['title'], list(movies_df.index)))
idx = movie_idx[title]
print(f"movie index for Jumanji: {idx}")

Wir erstellen eine Funktion, um die Empfehlungen für einen bestimmten Film zu erhalten

In [64]:
def find_similar_movies(movieTitle, n_recommendations=10):
    title = movie_finder(movieTitle)
    idx = movie_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations+1]
    similar_movies = [i[0] for i in sim_scores]
    print(f"because you watched {title}")
    print(movies_df['title'].iloc[similar_movies])

In [ ]:
find_similar_movies('Toy story', 3)

### **Schritt 7: Matrixfaktorisierung zur Dimensionalitätsreduktion**

Die Matrixfaktorisierung (MF) ist ein Verfahren der linearen Algebra, mit dem latente Merkmale entdeckt werden können, die den Interaktionen zwischen Nutzern und Filmen zugrunde liegen. Diese latenten Merkmale ermöglichen eine kompaktere Darstellung der Vorlieben der Nutzer und der Objektbeschreibungen. MF ist besonders nützlich für sehr spärliche Daten und kann die Qualität von Empfehlungen verbessern. Der Algorithmus funktioniert durch Faktorisierung der ursprünglichen Benutzer-Element-Matrix in zwei Faktormatrizen:

- Benutzer-Faktor-Matrix (n_Nutzer, k)
- Artikel-Faktor-Matrix (k, n_Einzelteile)

Wir reduzieren die Dimensionen unserer ursprünglichen Matrix auf „Geschmacks“-Dimensionen. Wir können die genaue Bedeutung der einzelnen latenten Merkmale *k* nicht interpretieren. Wir können uns jedoch mögliche Bedeutungen für sie vorstellen, z. B. könnte ein Merkmal für Nutzer stehen, die Comedy-Action-Filme aus den frühen 2000er Jahren mögen, während ein anderes für Bollywood-Filme aus den 90er Jahren stehen könnte.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, n_iter=10)
Q = svd.fit_transform(sparse_matrix.T)
Q.shape

In [ ]:
movie_id = 1
recommended_movies = find_recommended_movies(movieId=movie_id, sparseMatrix=Q.T, movieMapper=movie_mapper, invMovieMapper=inv_movie_mapper, metric='cosine', k=10)

print(f"for the movie {movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == movie_id].unique().item()}, the recommendations are:")
for id in recommended_movies:
    print(movie_ratings_df['title'].loc[movie_ratings_df['movieId'] == id].unique().item())

Die Ergebnisse sind ähnlich wie bei einem KNN-Modell, es scheinen durchweg Filme aus den 90er Jahren zu sein, einschließlich einiger früherer Kinderfilme